In [ ]:
from PIL import Image
from typing import Iterator
from collections import defaultdict, Counter 
import random 

from itertools import islice

### Open a 512*512 grayscale image 

In [ ]:
source=Image.open("../images/swim.512.png")
source

In [ ]:
data = bytearray(source.getdata())

In [ ]:
Image.frombytes('L',size=source.size, data=data)

### Compute the Marvov chain statistics

In [ ]:

counters = defaultdict(lambda: Counter())
a,b,c = None, None , None 
 

for pixel in data:
    if a is not None:
        counters[(a,b,c)][pixel]+=1

    a,b,c = b,c,pixel

counters = {**counters}

In [ ]:
for key, value in islice(counters.items(),10):
    print(f"{key}: {value}")

### Create a markov process that produces pixels

In [ ]:
rng = random.Random()
 
 
def process()->Iterator[int]:
    a,b,c, = random.choice(list(counters.keys()))
    while True:
        yield c
        counter = counters[(a,b,c)]
        choices, weights = zip(*counter.items())
        a=b
        b=c
        c=rng.choices(choices,weights)[0] 


In [ ]:
p= process()

In [ ]:
next(p)

### Use it to create an image

In [ ]:
def create():
    x,y= source.size
    b=bytes(islice(process(), x*y))
    return Image.frombytes('L',(x,y), b)


In [ ]:
create( )

### Can we do better? 

What if instead of operating on raster lines we operated on a hilbert curve?

In [ ]:
from hilbertcurve.hilbertcurve import HilbertCurve

In [ ]:
curve = HilbertCurve(p=9,n=2)


# pre-compute mapping of raster index to hilbert distance

def point_to_index(point):
    x,y=point 
    return y*512+x

def index_to_point(i):
    y,x = divmod(i,512)
    return [x,y]

distance_to_index = [
    point_to_index(curve.point_from_distance(distance))
    for distance in range(512*512)
]


index_to_distance=[
    curve.distance_from_point(index_to_point(index))
    for index in range(512*512)
]

In [ ]:
def to_hilbert(pixels):
     
    output = [0 for _ in range(512*512)]
    for i, pixel in enumerate(pixels):
        distance = index_to_distance[i] 
        assert distance<(512*512), distance
        assert output[distance] == 0 
        output[distance]=pixel 
    return output 

 

def to_raster(pixels):
    output = [0 for _ in range(512*512)]
    for distance, pixel in enumerate(pixels):
        
        i = distance_to_index[distance] 
        assert output[i] ==  0 
        output[i]=pixel 
    return output 




In [ ]:
# learn 
a,b,c = None, None, None
counters = defaultdict(lambda:Counter())
hilbert_pixels=to_hilbert(source.getdata())
for pixel in hilbert_pixels + hilbert_pixels[:3]:
    if a is not None:
        counters[(a,b,c)][pixel]+=1 
    a,b,c = b,c,pixel 

 
counters={**counters}
  
     

In [ ]:
def process(seed:str ):
    rng = random.Random(x=seed)
    a,b,c = rng.choice(list(counters.keys()))
    while True:
        yield c
        counter = counters[(a,b,c)]
        assert len(counter), (counter,a,b,c)
        choices, weights = zip(*counter.items())
        a=b
        b=c
        c=rng.choices(choices,weights)[0]


In [ ]:

hilbert_pixels = list(islice(process('some_seed'), 512*512))
cartesian_pixels = to_raster(hilbert_pixels)
created= Image.frombytes('L',(512,512), bytes(cartesian_pixels))
created

In [ ]:
from PIL import ImageFilter

In [ ]:
created.filter(ImageFilter.GaussianBlur(2))